<a href="https://colab.research.google.com/github/yuva23/Ds-python/blob/main/fraud_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Number of samples
n_samples = 1000

# Generate data
np.random.seed(0)
data = {
    'Transaction ID': np.arange(1, n_samples + 1),
    'User ID': np.random.randint(1, 200, n_samples),
    'Transaction Amount': np.random.uniform(5, 500, n_samples).round(2),
    'Transaction Date': pd.date_range(start='20230101', periods=n_samples, freq='T'),
    'Merchant Category': np.random.choice(['Grocery', 'Electronics', 'Clothing', 'Restaurant', 'Travel'], n_samples),
    'Method of Payment': np.random.choice(['Credit Card', 'Debit Card', 'PayPal'], n_samples),
    'Location': np.random.choice(['Online', 'In-store'], n_samples),
    'Device Type': np.random.choice(['Mobile', 'Desktop', 'Tablet'], n_samples),
    'Previous Transactions': np.random.poisson(5, n_samples),
    'Label': np.random.choice([0, 1], n_samples, p=[0.95, 0.05])  # 5% fraud rate
}

df = pd.DataFrame(data)

# Save to CSV
df.to_csv('transactions.csv', index=False)


In [ ]:
pip install numpy pandas tensorflow scikit-learn matplotlib


In [ ]:
import pandas as pd

data = pd.read_csv('transactions.csv')
# Assume 'label' column where 1 is fraudulent and 0 is non-fraudulent
X = data.drop('Label', axis=1)  # Features
y = data['Label']              # Labels


In [ ]:
# Select only numerical columns for scaling
numerical_cols = ['Transaction ID', 'User ID', 'Transaction Amount', 'Previous Transactions']
X_numerical = X[numerical_cols]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numerical)

In [ ]:
# If you need to keep the scaled data in the original DataFrame:
X[numerical_cols] = X_scaled

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten

model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_scaled.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    LSTM(50, return_sequences=True),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 3, 64)             192       
                                                                 
 max_pooling1d (MaxPooling1  (None, 1, 64)             0         
 D)                                                              
                                                                 
 lstm (LSTM)                 (None, 1, 50)             23000     
                                                                 
 flatten (Flatten)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                        

In [ ]:
model.fit(X_scaled, y, epochs=10, batch_size=64, validation_split=0.2)


Epoch 1/10
13/13 [==============================] - 4s 59ms/step - loss: 0.6266 - accuracy: 0.9150 - val_loss: 0.5347 - val_accuracy: 0.9450
Epoch 2/10
13/13 [==============================] - 0s 7ms/step - loss: 0.4576 - accuracy: 0.9513 - val_loss: 0.3322 - val_accuracy: 0.9450
Epoch 3/10
13/13 [==============================] - 0s 7ms/step - loss: 0.2715 - accuracy: 0.9513 - val_loss: 0.2294 - val_accuracy: 0.9450
Epoch 4/10
13/13 [==============================] - 0s 9ms/step - loss: 0.2141 - accuracy: 0.9513 - val_loss: 0.2452 - val_accuracy: 0.9450
Epoch 5/10
13/13 [==============================] - 0s 9ms/step - loss: 0.2121 - accuracy: 0.9513 - val_loss: 0.2394 - val_accuracy: 0.9450
Epoch 6/10
13/13 [==============================] - 0s 8ms/step - loss: 0.2087 - accuracy: 0.9513 - val_loss: 0.2313 - val_accuracy: 0.9450
Epoch 7/10
13/13 [==============================] - 0s 7ms/step - loss: 0.2064 - accuracy: 0.9513 - val_loss: 0.2317 - val_accuracy: 0.9450
Epoch 8/10
13/13 [=

In [ ]:
loss, accuracy = model.evaluate(X_scaled, y)
print(f'Accuracy: {accuracy*100:.2f}%')


32/32 [==============================] - 0s 4ms/step - loss: 0.2084 - accuracy: 0.9500
Accuracy: 95.00%


In [ ]:
model.save('fraud_detection_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
